In [35]:
import pdfplumber
import random
from os import listdir
from os.path import isfile, join
import json

debug=False

class Vorlesung:
    def __init__(self):
        self.tag = ""
        self.name = ""
        self.longname = ""
        self.dozent = ""
        self.raum = ""
        self.startBlock = ""
        self.endBlock = ""
        self.notes = ""

    def __str__(self):
        return (
            " "
            + self.name
            + ". "
            + self.dozent
            + ". "
            + self.raum
            + ". "
            + str(self.tag)
            + ". "
            + str(self.startBlock)
            + "-"
            + str(self.endBlock)
            + ". "
            + str(self.notes)
        )
def numberToDay(num):
    if abs(num-110)<5:
        return 1
    elif abs(num-248)<5:
        return 2
    elif abs(num-385)<5:
        return 3
    elif abs(num-523)<5:
        return 4
    elif abs(num-661)<5:
        return 5

def numberToBlockstart(num):
    if abs(num-111)<5:
        return 1
    elif abs(num-172)<5:
        return 2
    elif abs(num-232)<5:
        return 3
    elif abs(num-294)<5:
        return 4
    elif abs(num-353)<5:
        return 5
    elif abs(num-413)<5:
        return 6
    elif abs(num-473)<5:
        return 7
    elif abs(num-534)<5:
        return 8

def pageToTimetable(page):
    if debug:
        img=page.to_image()
        img.reset()
    titleCrop=page.crop((0,60,page.width,80))
    title=titleCrop.extract_text()
    rects =  [ x for x in page.rects
            if x["width"] > 100 and x["width"] < 200 and x["height"]>30]
    vls=[]
    for rect in rects:
        if debug:
            img.draw_rect(rect,stroke_width=4,stroke=(random.randint(0,255), random.randint(0,255), random.randint(0,255))) 
        box=(round(rect['x0']),round(rect['top']),round(rect['x1']),round(rect['bottom']))     
        cropt=page.crop(box, relative=True)
        text=cropt.extract_text().split()
        vl=Vorlesung()
        vl.startBlock=numberToBlockstart(box[1])
        vl.endBlock=numberToBlockstart(box[3])-1
        vl.tag=numberToDay(box[0])
        if text[0]==text[1]:
            vl.name=text[0]
            vl.dozent=text[2]+", "+text[3]
            
            if len(text)>5 and text[4]!=text[5]:
                vl.raum=text[4]+", "+text[5]
            else:
                vl.raum=text[4]
            if len(text)>6:
                vl.notes=text[6]
        else:
            vl.name=text[0]
            vl.dozent=text[1]
            vl.raum=text[2]
            if len(text)>3:
                vl.notes=text[3]
        if debug:
            print(vl)
        vls.append(vars(vl))
    
    if debug:
        display(img)
    return {title: vls} 


In [36]:
def parsePdf(path):
    pdf=pdfplumber.open(path)
    timetables={}
    for page in pdf.pages:
        if "Langname" in page.extract_text():
            pass
        else:
            timetables=timetables|pageToTimetable(page)
        if debug:
            print(page.page_number)

    if debug:
        print(timetables)
    return {list(timetables.keys())[0].split()[0]:timetables}



In [37]:
def parseDir(dir):
    onlyfiles = [f for f in listdir(dir) if isfile(join(dir, f))]
    timetables={}
    for file in onlyfiles:
        if debug:
            print(file)
        if ".pdf" ==file[-4:]:
            timetables=timetables|parsePdf(dir+"/"+file)
    return timetables

In [39]:
with open("timetables.json", "w") as outfile: 
    json.dump(parseDir("Stundenpläne"),outfile)